In [1]:
import weave
import datetime

weave.init_trace_client("timssweeney/weave_dev_1")

@weave.type()
class NumberHolder:
    val: int

@weave.op()
def add(a, b):
    return NumberHolder(a.val + b.val)

@weave.op()
def mult(a, b):
    return NumberHolder(a.val * b.val)

@weave.op()
def line(m, b, x):
    return add(mult(m, x), b)

start_time = datetime.datetime.now()
loop_count = 0
row_count = 0
for m in range(loop_count):
    saved_m = NumberHolder(m)
    weave.publish(saved_m)
    # print("HERE", saved_m)
    for b in range(loop_count):
        for x in range(loop_count):
            line(saved_m, NumberHolder(b), NumberHolder(x))
            row_count += 1
            elapsed = datetime.datetime.now() - start_time
    # print("ROWS", row_count, "Time", elapsed, "time/row", elapsed / row_count)

wandb: ERROR problem patching: monitor logging (via openai patch for example) is only supported with wandb client currently, auto-unpatching


ValueError: monitor logging (via openai patch for example) is only supported with wandb client currently

In [16]:
import clickhouse_connect

In [18]:
client = clickhouse_connect.get_client(host="localhost")

In [27]:
client.command("SHOW TABLES")

In [26]:
client.command("DROP TABLE objects")

In [28]:
command = """
    CREATE TABLE IF NOT EXISTS
    call_start (
        entity String,
        project String,
        id String,
        trace_id String NULL,
        parent_id String NULL,
        name String,
        start_datetime DateTime64(3),
        inputs Map(String, String),
        attributes Map(String, String),
        start_created_at DateTime64(3) DEFAULT now64(3),
    )
    ENGINE = MergeTree
    ORDER BY (entity, project, id)
"""

client.command(command)

In [31]:
command = """
    CREATE TABLE IF NOT EXISTS
    call_end (
        entity String,
        project String,
        id String,
        end_datetime DateTime64(3),
        
        outputs Map(String, String),
        summary Map(String, String),
        exception String NULL,
        
        end_created_at DateTime64(3) DEFAULT now64(3),
    )
    ENGINE = MergeTree
    ORDER BY (entity, project, id)
"""
client.command(command)

In [38]:
command = """
    INSERT INTO call_start (
        entity,
        project,
        id,
        trace_id,
        parent_id,
        name,
        start_datetime,
        inputs,
        attributes
    ) VALUES (
        'entity',
        'project',
        'id',
        NULL,
        NULL,
        'name',
        now64(3),
        {'a': 'a'},
        {'b': 'b'},
    )
"""
client.command(command)

DatabaseError: :HTTPDriver for http://localhost:8123 returned response code 400)
 Code: 62. DB::Exception: Cannot parse expression of type Map(String, String) here: {'b': 'b'},
    )
: While executing ValuesBlockInputFormat. (SYNTAX_ERROR) (version 24.2.1.1007 (official build))


In [313]:
res = client.command("SHOW TABLES")
if type(res) == str:
    table_names = res.split("\n")
    for table_name in table_names:
        if not table_name.startswith("."):
            print("deleting " + table_name)
            client.command("DROP TABLE IF EXISTS " + table_name)
    print(client.command("SHOW TABLES"))



client.command(
    """
CREATE TABLE IF NOT EXISTS
objects_raw (
    entity String,
    project String,
    name String,
    version_hash String,
    created_datetime DateTime64(3),

    type_dict_dump String,
    b64_file_map Map(String, String),
    metadata_dict_dump String,

    db_row_created_at DateTime64(3) DEFAULT now64(3)

)
ENGINE = MergeTree
ORDER BY (entity, project, name, version_hash)
"""
)

client.command(
    """
CREATE TABLE IF NOT EXISTS
objects_deduplicated (
    entity String,
    project String,
    name String,
    version_hash String,
    orig_created_datetime SimpleAggregateFunction(min, DateTime64(3)),

    type_dict_dump SimpleAggregateFunction(any, String),
    b64_file_map SimpleAggregateFunction(any, Map(String, String)),
    metadata_dict_dump SimpleAggregateFunction(any, String),
)
ENGINE = AggregatingMergeTree
ORDER BY (entity, project, name, version_hash)
"""
)


client.command("""
CREATE MATERIALIZED VIEW objects_deduplicated_view TO objects_deduplicated
AS
SELECT
    entity,
    project,
    name,
    version_hash,
    minSimpleState(created_datetime) as orig_created_datetime,

    anySimpleState(type_dict_dump) as type_dict_dump,
    anySimpleState(b64_file_map) as b64_file_map,
    anySimpleState(metadata_dict_dump) as metadata_dict_dump
    
FROM objects_raw
GROUP BY entity, project, name, version_hash
""")



# The following view is just to workout version indexing and latest keys
client.command("""
CREATE VIEW objects_versioned
AS
SELECT
    entity,
    project,
    name,
    version_hash,
    min(orig_created_datetime) as created_datetime,
    any(type_dict_dump) as type_dict_dump,
    any(b64_file_map) as b64_file_map,
    any(metadata_dict_dump) as metadata_dict_dump,
    row_number() OVER w1 as version_index,
    1 == count(*) OVER w1 as is_latest
FROM objects_deduplicated
GROUP BY entity, project, name, version_hash
WINDOW
    w1 AS (PARTITION BY entity, project, name ORDER BY min(orig_created_datetime) ASC Rows BETWEEN CURRENT ROW AND 1 FOLLOWING)
""")


for v in range(5):
    for dup in range(2):
        for i in ['a', 'b']:
            print("insert")
            client.insert("objects_raw", 
                          # data=[["entity", "project", "name" + i, "version_hash" + str(v), datetime.datetime.now()]], 
                          # column_names=["entity", "project", "name", "version_hash", "created_datetime"])
                          data=[["entity", "project", "name" + i, "version_hash" + str(v),  datetime.datetime.now(), "type_dict_dump", {'a': 'a'}, "metadata_dict_dump"]], 
                          column_names=["entity", "project", "name", "version_hash", "created_datetime", "type_dict_dump", "b64_file_map", "metadata_dict_dump"])


res = client.query_df("""
SELECT
*
    
FROM objects_versioned

""", use_none=True)

print(res)


# print(res)

    # count(*) OVER (PARTITION BY name ORDER BY created_datetime ASC Rows BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as version_index,
# 
# client.command(
#     """
# CREATE TABLE IF NOT EXISTS
# objects_first (
#     entity String,
#     project String,
#     name String,
#     version_hash String,
#     created_datetime DateTime64(3) DEFAULT now64(3),
# )
# ENGINE = MergeTree
# ORDER BY (entity, project, name, version_hash)
# """
# )




res = client.command("SHOW TABLES")
table_names = res.split("\n")
for table_name in table_names:
    if not table_name.startswith("."):
        res = client.command("DESCRIBE TABLE " + table_name)
        print(table_name)
        print("-----")
        print("\t".join(res))
        print("")

deleting objects_deduplicated
deleting objects_deduplicated_view
deleting objects_raw
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
   entity  project   name   version_hash                 created_datetime  \
0  entity  project  namea  version_hash0 2024-02-21 19:05:53.701000-08:00   
1  entity  project  namea  version_hash1 2024-02-21 19:05:53.724000-08:00   
2  entity  project  namea  version_hash2 2024-02-21 19:05:53.941000-08:00   
3  entity  project  namea  version_hash3 2024-02-21 19:05:54.864000-08:00   
4  entity  project  namea  version_hash4 2024-02-21 19:05:54.889000-08:00   
5  entity  project  nameb  version_hash0 2024-02-21 19:05:53.708000-08:00   
6  entity  project  nameb  version_hash1 2024-02-21 19:05:53.730000-08:00   
7  entity  project  nameb  version_hash2 2024-02-21 19:05:53.946000-08:00   
8  entity  project  nameb  version_hash3 2024-02-21 19:05:54.871000-08:00   
9  e

In [124]:
[1] * 10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [107]:
res = client.insert(
    "calls_raw",
    data=[[
        "entity",
        "project",
        "id",
        "trace_id",
        "parent_id",
        "name",
        datetime.datetime.now(),
        "{}",
        "{}",
        []
    ]],
    column_names=[
        "entity",
        "project",
        "id",
        "trace_id",
        "parent_id",
        "name",
        "start_time",
        "attributes_dump",
        "inputs_dump",
        "input_refs"
    ],
    # column_type_names=all_obj_column_type_names_with_defaults_nullable,
)
res.summary

{'read_rows': '2',
 'read_bytes': '315',
 'written_rows': '2',
 'written_bytes': '309',
 'total_rows_to_read': '0',
 'result_rows': '2',
 'result_bytes': '309',
 'elapsed_ns': '5053500',
 'query_id': '0d78a67b-0218-49f5-a85b-5440ac562abc'}

In [116]:
res = client.query("""
SELECT
    entity,
    project,
    id,
    any(trace_id) as trace_id,
    any(parent_id) as parent_id,
    any(name) as name,
    any(start_time) as start_time,
    any(attributes_dump) as attributes_dump,
    any(inputs_dump) as inputs_dump,
    array_concat_agg(input_refs) as input_refs,
    any(end_time) as end_time,
    any(outputs_dump) as outputs_dump,
    any(summary_dump) as summary_dump,
    any(exception) as exception,
    array_concat_agg(output_refs) as output_refs
FROM calls_merged
GROUP BY entity, project, id
""", use_none=True)
print(res.summary)
res.result_rows

{'read_rows': '2', 'read_bytes': '350', 'written_rows': '0', 'written_bytes': '0', 'total_rows_to_read': '2', 'result_rows': '1', 'result_bytes': '19966', 'elapsed_ns': '5334917', 'query_id': '09464287-f88a-469b-84a1-8ba29ab478f2'}


[('entity',
  'project',
  'id',
  'trace_id',
  'parent_id',
  'name',
  datetime.datetime(2024, 2, 21, 13, 34, 49, 93000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')),
  '{}',
  '{}',
  [],
  None,
  None,
  None,
  None,
  [])]

In [ ]:
res = client.insert(
    "calls_raw",
    data=[[
        "entity",
        "project",
        "id",
        "trace_id",
        "parent_id",
        "name",
        datetime.datetime.now(),
        "{}",
        "{}",
        []
    ]],
    column_names=[
        "entity",
        "project",
        "id",
        "trace_id",
        "parent_id",
        "name",
        "start_time",
        "attributes_dump",
        "inputs_dump",
        "input_refs"
    ],
    # column_type_names=all_obj_column_type_names_with_defaults_nullable,
)
res.summary